In [1]:
import pandas as pd
from common import clear_players
from IPython.display import display, HTML
from tqdm.notebook import trange, tqdm

# Load Data

In [2]:
matches = pd.read_csv("data/orginal/match.csv")
players = pd.read_csv("data/orginal/players.csv")
players_cleared = clear_players(players)

# Generate dataset

## Calculate

In [9]:
NUM_MATCHES = 10_000

matches_latest = matches.sort_values(by='match_id', ascending=False)

result_df = pd.DataFrame()

num_exported = 0

with tqdm(total=NUM_MATCHES) as pbar: 
    for i, values in matches_latest.iterrows():
        match_id = values['match_id']
        
        players_list = players_cleared[(players_cleared['match_id'] == match_id) & (players_cleared['account_id'] != 0)].sort_values('player_slot')
        hist_players = players_cleared[(players_cleared['match_id'] < match_id) & (players_cleared['account_id'].isin(players_list['account_id']))]
        
        stats = hist_players.drop(['match_id', 'hero_id', 'player_slot'], axis=1).groupby('account_id').mean()
        
        if len(stats) < 6:
            continue
        
        entry = {
            "rad_won": values['radiant_win']
        }
        
        for acc_id, acc_means in stats.iterrows():
            slot = players_list[players_list['account_id'] == acc_id]['player_slot'].iloc[0]
            for key, val in acc_means.iteritems():
                entry[f"{slot}_{key}"] = val
                            
        result_df = result_df.append(entry, ignore_index = True)
        
        num_exported += 1
        pbar.update(1)
        
        if num_exported >= NUM_MATCHES:
            break

KeyboardInterrupt: 

## Save to file

In [73]:
# sort columns
def column_sort(x):
    if x == 'rad_won':
        return 999999999
    else:
        return int(x.split("_")[0])

result_df = result_df.reindex(columns=sorted(result_df.columns,key=column_sort))
result_df.to_csv("data/generated/dataset_min6.csv")